In [1]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

In [2]:
print("TensorFlow version: {}".format(tf.__version__))
print("Eager execution: {}".format(tf.executing_eagerly()))

TensorFlow version: 2.2.0
Eager execution: True


In [3]:
import numpy as np
import sklearn
from sklearn import preprocessing

In [6]:
mode = "gat"
hidden_units = "16,16"
heads = "8,8,1"
epochs = 5
lr = 0.1
dropout = 0.2
file_dir = '/Users/lsj/Downloads/DeepInf-homework/weibo'
batch = 1024
train_ratio = 75
vaild_ratio = 12.5
instance_normalization = True
use_vertex_feature = True


In [7]:
def load_w2v_feature(file, max_idx=0):
    with open(file, "rb") as f:
        nu = 0
        for line in f:
            content = line.strip().split()
            nu += 1
            if nu == 1:
                n, d = int(content[0]), int(content[1])
                feature = [[0.] * d for i in range(max(n, max_idx + 1))]
                continue
            index = int(content[0])
            while len(feature) <= index:
                feature.append([0.] * d)
            for i, x in enumerate(content[1:]):
                feature[index][i] = float(x)
    for item in feature:
        assert len(item) == d
    return np.array(feature, dtype=np.float32)

In [16]:
vertices = np.load(os.path.join(file_dir, "vertex_id.npy"))
embedding_path = os.path.join(file_dir, "deepwalk.emb_64")
max_vertex_idx = np.max(vertices)
embedding = load_w2v_feature(embedding_path, max_vertex_idx)
print(embedding.shape())

TypeError: 'tuple' object is not callable

In [18]:
embedding.shape[0]


1784024

In [ ]:
graphs = np.load(os.path.join(file_dir, "adjacency_matrix.npy")).astype(np.float32)
# self-loop trick, the input graphs should have no self-loop
identity = np.identity(graphs.shape[1])
graphs += identity
graphs[graphs != 0] = 1.0
if model == "gat":
   graphs = graphs.astype(np.dtype('B'))
elif model == "gcn":
    # normalized graph laplacian for GCN: D^{-1/2}AD^{-1/2}
    for i in range(len(graphs)):
        graph = graphs[i]
        d_root_inv = 1. / np.sqrt(np.sum(graph, axis=1))
        graph = (graph.T * d_root_inv).T * d_root_inv
        graphs[i] = graph
else:
    raise NotImplementedError

# wheather a user has been influenced
# wheather he/she is the ego user
influence_features = np.load(
                os.path.join(file_dir, "influence_feature.npy")).astype(np.float32)

labels = np.load(os.path.join(file_dir, "label.npy"))

vertices = np.load(os.path.join(file_dir, "vertex_id.npy"))

graphs, influence_features, labels,vertices = sklearn.utils.shuffle(
                        graphs, influence_features,
                        labels, vertices,
                        random_state=47)

vertex_features = np.load(os.path.join(file_dir, "vertex_feature.npy"))
vertex_features = preprocessing.scale(vertex_features)

embedding_path = os.path.join(file_dir, "deepwalk.emb_64")
max_vertex_idx = np.max(vertices)
embedding = load_w2v_feature(embedding_path, max_vertex_idx)



In [ ]:
vertices_inputs = keras.Input(shape=(None,))
feature_inputs = keras.Input(shape=(None,))
graph_inputs = keras.Input(shape=(None,))

##todo: emb init
#emb = keras.layers.Embedding(embedding.shape[0],embedding.shape[1],embeddings_initializer=xxxx)
##todo: instance normalization
##emb = tfa.layers.InstanceNormalization(emb)
#x = keras.layers.Concatenate(feature_inputs,emb)
x = feature_inputs
#x = keras.layers.Embedding(vertex_features.shape(0),ertex_features.shape(1), embeddings_initializer=xxx)

##todo: multi head
x = keras.layers.AdditiveAttention(x,graph_inputs) 
outputs = keras.layers.Softmax(x)


In [ ]:
class CustomModel(keras.Model):
    def train_step(self, data):
        # Unpack the data. Its structure depends on your model and
        # on what you pass to `fit()`.
        #print(data)
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)  # Forward pass
            # Compute the loss value
            # (the loss function is configured in `compile()`)
            loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

        # Compute gradients
        trainable_vars = self.trainable_variables
        trainable_vars = filter(lambda x: 'embedding' not in x.name ,trainable_vars)
        for v in trainable_vars:
           print(v.name)
        gradients = tape.gradient(loss, trainable_vars)
        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        # Update metrics (includes the metric that tracks the loss)
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

In [ ]:
model = CustomModel(
    inputs=[vertices_inputs, feature_inputs, graph_inputs],
    outputs=[outputs],
)

In [0]:
## todo: tf nllloss?
model.compile(
    optimizer=keras.optimizers.Adagrad(lr=lr),
    loss=keras.losses.BinaryCrossentropy(from_logits=True),
    metrics=?
)


In [ ]:
model.fit(
    {"vertices": vertices , "feature":influence_features , "graph": graphs},
    {"output": outputs},
    epochs=2,
    batch_size=32,
)